In [1]:
#CHANGE THE PATH TO THE PATH AT YOUR PC!
path = "C:\\Users\\user\\Desktop\\TUe\\Topological\\Project\\Geolife Trajectories 1.3\\Data\\"
CITY_BORDERS = [40.19, 39.65, 115.98, 116.74]
crop_to_borders = True
THRESHOLD_VERTICES = 50
THRESHOLD_EDGES = 4
GRID_SHAPE = (40, 40)

import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from Graph import *
from Grid import *

In [2]:
df = pd.read_csv(".\\presets\\very_short.csv")

In [3]:
def grid_graphify(df: pd.DataFrame, crop_to_borders: bool, grid_shape: tuple, 
                  THRESHOLD_VERTICES: int, THRESHOLD_EDGES: int, CITY_BORDERS: list):
    '''#How many locations do we need for a cell to be classified as a vertex? Enter the threshold:'''

    #(possibly) cropping the data to the city borders:
    if crop_to_borders:
        mask_north = df['latitude'] <= CITY_BORDERS[0]
        mask_south = df['latitude'] >= CITY_BORDERS[1]
        mask_east = df['longitude'] <= CITY_BORDERS[3]
        mask_west = df['longitude'] >= CITY_BORDERS[2]
        mask_all = mask_north & mask_south & mask_east & mask_west
        df = df[mask_all].reset_index(drop=True)
    
    #Grid creation example:
    grid = Grid(grid_shape[0], grid_shape[1]) 
    print(grid)
    print(grid.cell_list[0].number_of_locations)
    
    
    #This will perform some expensive computation. Around 7s on the 160k dataset
    df['column_number'] = df['latitude'].apply(latitude_to_row, 
                                               args=(grid.latitude_cells_nr, 
                                                     grid.latitude_cell_length,
                                                     CITY_BORDERS))

    df['row_number'] = df['longitude'].apply(longitude_to_column,
                                             args=(grid.longitude_cells_nr,
                                                   grid.longitude_cell_length,
                                                   CITY_BORDERS))

    df['cell_number'] = df.apply(lambda x: row_column_to_cell_number(x['column_number'], 
                                                                     x['row_number'],
                                                                     grid.longitude_cells_nr),axis = 1)

    df = df.drop(['column_number', 'row_number'], axis=1)
    df = df.dropna(subset=['cell_number'])
    df = df.reset_index(drop=True)
    
    print(df.shape)
    
    #This actually performs the large computation:
    grid.feed_list_of_cell_numbers(list(df.cell_number))

    #And this just switches some flags around:
    grid.vertexize(THRESHOLD_VERTICES)
    
    #By grouping per trajectory, we can find the unique vertices each trajectory has traversed!
    df_grouped = df.groupby('trajectory')
    print(grid.cell_list[12].is_vertex)
    
    #{(213, 7): 5, ...} meaning there are 5 trajectories that went from cell 213 to cell 7 immediately
    edges_dict = find_subsequent_vertex_pairs(df_grouped, grid)
    
    #Same as above, but filtered by a threshold
    edges = [key for key in edges_dict if edges_dict[key] > THRESHOLD_EDGES]
    print(edges[:5])
    
    g = rav_graph_to_sven_graph_2(grid, edges)
    
    g.remove_unconnected_vertices()
    
    return g

In [4]:
g = grid_graphify(df, True, GRID_SHAPE, THRESHOLD_EDGES, THRESHOLD_VERTICES, CITY_BORDERS)
g.save('random_graph_4')

Grid of side length (latitude) 0.0135 and (longitude) 0.019 and total number of cells: 1600
0
(90386, 9)
False
[(1019, 1020), (1019, 1018), (1020, 1019)]


Determining whether the cell is or is not a vetex - for all cells in the grid:

In [ ]:
# def plot_cell_containment(grid: Grid, threshold: int, bins = 20):
#     ''''''
#     wow = pd.Series([grid.cell_list[i].number_of_locations for i in range(grid.latitude_cells_nr*grid.longitude_cells_nr)])
#     wow = pd.Series([wow[i] for i in range(len(wow)) if wow[i] > threshold])
#     ax = wow.plot(kind='hist', bins=bins)
#     ax.set_xlabel("# GPS locations per cell of the grid")
#     ax.set_ylabel("# cells with such # locations")
    
# plot_cell_containment(grid, threshold=THRESHOLD_VERTICES)

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))
ax.scatter(df.latitude, df.longitude, zorder=1, alpha= 0.2, c='b', s=10)

# for edge in g.edges:
#     lat1 = min(edge.vertices).latitude
#     long1 = min(edge.vertices).longitude
    
#     lat2 = max(edge.vertices).latitude
#     long2 = max(edge.vertices).longitude
    
#     plt.plot([lat1, lat2], [long1, long2], marker = 'o')

# star_vertices = [vertex for vertex in graph.vertices if len(vertex.edges) == 1]
# for vertex in star_vertices:
#     ax.plot(vertex.latitude, vertex.longitude, markersize=50)
    
ax = plt.gca()
#ax.set_xlim([40.0035, 40.0115])
#ax.set_ylim([116.3175, 116.3245])
print(f"reduced from {len(df)} to {len(g.vertices)} points")